In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
## Tool for Wikipedia API

from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
tool_wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
## Tool for Arxiv API

from langchain.tools import ArxivQueryRun
from langchain_community.utilities import  ArxivAPIWrapper

api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
tool_arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [4]:
## Custom Tool for langchain docs

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

loader = WebBaseLoader(web_path='https://python.langchain.com/v0.2/docs/concepts/')
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = splitter.split_documents(docs)

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
db = FAISS.from_documents(documents, embeddings)
retirver = db.as_retriever()


/home/sidd/miniconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
retirver.invoke("agents")

[Document(page_content="You can therefore do:# Initialize a toolkittoolkit = ExampleTookit(...)# Get list of toolstools = toolkit.get_tools()Agents\u200bBy themselves, language models can't take actions - they just output text.\nA big use case for LangChain is creating agents.\nAgents are systems that use an LLM as a reasoning enginer to determine which actions to take and what the inputs to those actions should be.\nThe results of those actions can then be fed back into the agent and it determine whether more actions are needed, or whether it is okay to finish.LangGraph is an extension of LangChain specifically aimed at creating highly controllable and customizable agents.\nPlease check out that documentation for a more in depth overview of agent concepts.There is a legacy agent concept in LangChain that we are moving towards deprecating: AgentExecutor.\nAgentExecutor was essentially a runtime for agents.", metadata={'source': 'https://python.langchain.com/v0.2/docs/concepts/', 'title

In [6]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retirver, "Langchain", "Search for concepts in the Langchain documentation. For any questions about Langchain, you must use this tool!")

In [7]:
tools = [retriver_tool, tool_wiki, tool_arxiv]

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [9]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
## Agent

from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [11]:
## Agent Executer

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, prompt=prompt, verbose=True)

In [12]:
agent_executor.invoke({"input": "what are agents in Langchain?"})



> Entering new AgentExecutor chain...

Invoking: `Langchain` with `{'query': 'What are agents in Langchain?'}`


You can therefore do:# Initialize a toolkittoolkit = ExampleTookit(...)# Get list of toolstools = toolkit.get_tools()Agents​By themselves, language models can't take actions - they just output text.
A big use case for LangChain is creating agents.
Agents are systems that use an LLM as a reasoning enginer to determine which actions to take and what the inputs to those actions should be.
The results of those actions can then be fed back into the agent and it determine whether more actions are needed, or whether it is okay to finish.LangGraph is an extension of LangChain specifically aimed at creating highly controllable and customizable agents.
Please check out that documentation for a more in depth overview of agent concepts.There is a legacy agent concept in LangChain that we are moving towards deprecating: AgentExecutor.
AgentExecutor was essentially a runtime for agents.

{'input': 'what are agents in Langchain?',
 'output': "Langchain Agents are systems that use LLMs (Large Language Models) for reasoning and decision-making to determine which actions to take based on user inputs. They use these LLMs to figure out what actions to take and what inputs to use for those actions. The results of these actions can then be fed back into the agent to decide if further actions are needed or if the task is complete. \n\nLet me know if you'd like to explore a specific aspect of Langchain Agents or have any other questions. \n"}